In [1]:
# Utils packages
import ast
from pathlib import Path
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [2]:
N_SAMPLE = 0 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [3]:
def random_hentai(_: int) -> pd.Series:
    return pd.Series(Utils.get_random_hentai().dictionary(Option.all()))

In [4]:
# Query the nhentai.net API to get N number of random hentai
hentais_df = pd.Series(range(N_SAMPLE))
hentais_df = hentais_df.apply(random_hentai)

# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "metadata.csv"

# Create a CSV file with a header only if the file does not exist
if not metadata_path.is_file():
    hentais_df.to_csv(metadata_path, header="column_names")
else:
    hentais_df.to_csv(metadata_path, mode="a", header=False)

print("Number of collected hentais: ", len(hentais_df))

ipykernel_launcher:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
Number of collected hentais:  0


## Read dataset file

In [5]:
hentais_df = pd.read_csv(
    Path(DATA_PATH) / "metadata.csv",
    converters={
        column_name: ast.literal_eval
        for column_name in ["tag", "group", "parody", "character", "artist", "category", "image_urls"]
    }
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
0,212548,Syoku,NaN,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33
1,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,NaN,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166
2,75495,SHIRONA HAZARD,NaN,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23
3,229284,2205-nen no Afterglow,NaN,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],[touken ranbu],[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47
4,99262,moon marguerite,NaN,https://nhentai.net/g/99262,https://nhentai.net/api/gallery/99262,635686,1404000355,138,"[stockings, paizuri, lingerie, sole female, so...",[meltdown comet],[touhou project],[patchouli knowledge],['japanese'],[yukiu con],[doujinshi],https://t.nhentai.net/galleries/635686/cover.jpg,https://t.nhentai.net/galleries/635686/thumb.jpg,"[https://i.nhentai.net/galleries/635686/1.jpg,...",26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,191631,Kobayashichi no Inu Dragon,NaN,https://nhentai.net/g/191631,https://nhentai.net/api/gallery/191631,1046505,1491184635,712,"[bestiality, big breasts, inverted nipples, mo...",[zensoku rider],[kobayashi-san-chi no maid dragon],"[kobayashi, tohru]",['japanese'],[tenzen miyabi],[doujinshi],https://t.nhentai.net/galleries/1046505/cover.jpg,https://t.nhentai.net/galleries/1046505/thumb.jpg,[https://i.nhentai.net/galleries/1046505/1.jpg...,10
538,325513,Ikumi-chan Niku Niku 2,NaN,https://nhentai.net/g/325513,https://nhentai.net/api/gallery/325513,1712669,1597933877,1387,"[sole male, big breasts, collar, dark skin, so...",[amazake hatosyo-ten],[shokugeki no soma],"[souma yukihira, ikumi mito]","['chinese', 'translated']",[yoshu ohepe],[doujinshi],https://t.nhentai.net/galleries/1712669/cover.jpg,https://t.nhentai.net/galleries/1712669/thumb.jpg,[https://i.nhentai.net/galleries/1712669/1.jpg...,30
539,150615,Usamimi Danshi Koukousei ga Masaka Mob ni Okas...,NaN,https://nhentai.net/g/150615,https://nhentai.net/api/gallery/150615,868662,1449593893,36,"[anal, bunny boy, males only, muscle, yaoi, sw...",[],[free],"[haruka nanase, makoto tachibana]",['japanese'],[],[doujinshi],https://t.nhentai.net/galleries/868662/cover.jpg,https://t.nhentai.net/galleries/868662/thumb.jpg,"[https://i.nhentai.net/galleries/868662/1.jpg,...",31
540,41682,Shirley to Lucchini no Gohoushi Hon,NaN,https://nhentai.net/g/41682,https://nhentai.net/api/gallery/41682,199951,1403989884,54,"[big breasts, group, lolicon]",[atsuya kougyou],[strike witches],"[francesca lucchini, charlotte e. yeager, shir...",['japanese'],[kaisen chuui],[doujinshi],https://t.nhentai.net/galleries/199951/cover.jpg,https://t.nhentai.net/galleries/199951/thumb.jpg,"[https://i.nhentai.net/galleries/199951/1.jpg,...",25


## Download images

In [14]:
def download_hentai(hentai: pd.Series) -> Hentai:
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / str(hentai.id)

    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        hentai = Hentai(hentai.id)
        hentai.download(hentai_path, progressbar=True)
        return True

    return False

In [15]:
hentais_df[hentais_df.apply(download_hentai, axis=1)]

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
